# Speadsheet calculator
## Extracts data and formats it into an excel sheet
### Jeffrey Zhou
### 21/11/2022

In [1]:
import pymrio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
'''
Data setup --> imported from Z_analysis
Note: this will take a while to run. Something like 3 mins
'''

exio3 = pymrio.parse_exiobase3(path='/Users/jeffreyzhou/Desktop/MRIO/Data/IOT_2019_pxp.zip')
all_regions = exio3.get_regions()
all_sectors = exio3.get_sectors()

#include all missing calculations with calc_all()
exio3.calc_all()

#US Z matrix - transaction matrix
us_z = exio3.Z.aggregate('US')

#US Y matrix - final demand matrix
us_y = exio3.Y.aggregate('US')

trade_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    trade_matrix_dict[region] = us_z.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200

demand_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    demand_matrix_dict[region] = us_y.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200

trade_calcvalues_dict = {}
    
for region in all_regions:
    tm = trade_matrix_dict[region]
    d = {}
    for sector in all_sectors:
        d[sector] = tm[sector].values.sum()
    in_series = pd.Series(data = d, index = all_sectors)
    trade_calcvalues_dict[region] = in_series

US_total_trade = {}

for sector in all_sectors:
    in_dict = {}
    for region in all_regions:
        if region == 'US': # comment out this line if including domestic inputs
            continue
        in_dict[region] = trade_calcvalues_dict[region][sector]
    US_total_trade[sector] = in_dict

In [45]:
df = pd.DataFrame(data=all_sectors)

In [25]:
# Save function to add columns to the spreadsheet

def save(database, lst, label):
    database[label] = lst
    return database

In [29]:
#function that plots graphs of specific width and height.

def plot_func(x, y, xlabel, ylabel, title):
    fig = plt.figure(figsize=(16, 5))
    ax = fig.add_subplot(111)
    ax.bar(x,y)
    fig.suptitle(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=18)
    plt.ylabel(ylabel, fontsize=16)

    
    plt.show()

## To add more categories to the spreadsheet, just extract an array of length == all_sectors and call the save() function

In [47]:
# Consumption impacts (% of consumption that come from imports)
imports = exio3.impacts.D_imp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)']
consumption = exio3.impacts.D_cba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)']
    
    
imports = imports.to_dict()
consumption = consumption.to_dict()

impacts_percent= {}
count = 0
for sector in all_sectors:
    d = consumption[sector]
    n = imports[sector]
    if d == 0:
        count += 1
        if n != 0:
            print("ERROR")
        impacts_percent[sector] = 'NA'
        continue
    percent = n / (d + n)
    impacts_percent[sector] = percent
    
x, y = zip(*impacts_percent.items())
df = save(df, y, "Consumption impacts (% of consumption GHG that come from imports)")

In [48]:
# Total GHG from imports
y2 = exio3.impacts.D_imp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y2, "Total GHG from imports")

In [49]:
# Total GHG from exports
y3 = exio3.impacts.D_exp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y3, "Total GHG from exports")

In [46]:
# Total GHG from consumption (Imports and domestic)
y4 = exio3.impacts.D_cba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df,y4, "Total GHG from consumption (Imports and domestic)")

In [50]:
# Total GHG from production (Domestic)
y5 = exio3.impacts.D_pba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y5, "Total GHG from production (Domestic)")

In [51]:
# Run to check numbers in database before exporting spreadsheet
df

,sector,Total GHG from consumption (Imports and domestic),Consumption impacts (% of consumption GHG that come from imports),Total GHG from imports,Total GHG from exports,Total GHG from production (Domestic)
0,Paddy rice,3.629765e+09,0.022168,8.228870e+07,4.558772e+09,2.982942e+10
1,Wheat,9.294773e+08,0.050546,4.948254e+07,1.068413e+10,2.256458e+10
2,Cereal grains nec,1.708762e+09,0.03702,6.568953e+07,2.072166e+10,5.302156e+10
3,"Vegetables, fruit, nuts",1.845900e+10,0.381537,1.138757e+10,1.914432e+09,8.025542e+09
4,Oil seeds,1.871437e+08,0.106301,2.225989e+07,1.145297e+10,1.601793e+10
...,...,...,...,...,...,...
195,Membership organisation services n.e.c. (91),3.299288e+10,0.121493,4.562761e+09,3.262627e+07,2.002287e+09
196,"Recreational, cultural and sporting services (92)",3.073611e+10,0.16426,6.040996e+09,1.524939e+08,6.049135e+09
197,Other services (93),4.547095e+10,0.142118,7.532801e+09,1.738770e+07,2.539353e+09
198,Private households with employed persons (95),1.487312e+09,0.481842,1.383072e+09,2.011200e+08,2.764586e+08


In [52]:
# Exports spreadsheet
name_of_file = 'Impacts_spreadsheet_US_2019' # Change this string to desired filename
df.to_excel(f'{name_of_file}.xlsx')